In [1]:
!python --version

Python 3.10.14


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import glob
import pandas as pd
from tqdm.autonotebook import tqdm
import matplotlib.dates as mdates
from itertools import islice
from collections import defaultdict
from datetime import datetime
import numpy as np

/tmp/ipykernel_3772999/1990381454.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from autogluon.timeseries.metrics import TimeSeriesScorer
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [4]:
TimeSeriesScorer.greater_is_better_internal = True

class NRMSE(TimeSeriesScorer):
   greater_is_better_internal = True
   optimum = 0.0

   def compute_metric(self, data_future, predictions, target, **kwargs):
      return np.sqrt(np.mean(np.square(data_future[target] - predictions["mean"]))) / data_future[target].mean()

In [5]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples

In [ ]:
def forecast_building(df):
    torch.cuda.empty_cache()
    # Set numerical columns as float32
    for col in df.columns:
        # Check if column is not of string type
        if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
            df[col] = df[col].astype('float32')
    
    # Create the Timeseries Dataframe
    dataset = TimeSeriesDataFrame(df.reset_index())

    backtest_dataset = dataset
    prediction_length = 24  # Define your prediction length. We use 24 here since the data is of hourly frequency
    num_samples = 100

    train_data, test_data = backtest_dataset.train_test_split(prediction_length)


    predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(
    train_data,
    hyperparameters={
        "Chronos": {
            "model_path": "small",
            "batch_size": 32,
            "device": "auto",
            "context_length": 168
        }},
    skip_model_selection=True,
    verbosity=0)
    predictions = predictor.predict(train_data)  
    agg_metrics = predictor.evaluate(backtest_dataset, metrics=["RMSE", "MSE", "MAE", "MSE", "MAPE", "SMAPE", NRMSE(), "SQL"])


    res_all = pd.DataFrame(test_data[test_data.index.isin(predictions.index)].target)
    res_all.columns = ['y_true']
    res_all.insert(1, 'y_pred', predictions['mean'])
    res_all_df = res_all.reset_index().drop('item_id', axis = 1).sort_values('timestamp')

    return res_all_df, agg_metrics


In [ ]:
def process_building(df): 
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    # print(input_data)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)

    res, agg_metrics = forecast_building(windows_all_df)
    return res, agg_metrics

In [ ]:
def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    res_all = []
    agg_metrics_all = []
    
    i = 0
    for building_name in df.columns:
        print(datetime.now(), i, '/', len(df.columns), building_name, flush=True)
        df1 = df[[building_name]]

        res, agg_metrics = process_building(df1)
        res['building'] = building_name
        res['filename'] = filename
        res_all.append(res)
        
        agg_metrics_df = pd.DataFrame([agg_metrics])
        agg_metrics_df.insert(0, 'building', building_name)
        agg_metrics_df.insert(0, 'filename', filename)
        agg_metrics_all.append(agg_metrics_df)

        i += 1
        if i % 5 == 0:
            print(datetime.now(), 'Saving...')
            res_all_df = pd.concat(res_all).round(6)
            res_all_df = res_all_df.reset_index()
            res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
            res_all_df.to_csv(f'forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

            agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)            
            agg_metrics_all_df.to_csv(f'results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)            
    
    
    res_all_df = pd.concat(res_all).round(6)
    res_all_df = res_all_df.reset_index()
    res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
    res_all_df.to_csv(f'forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

    agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)   
    agg_metrics_all_df.to_csv(f'results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)                

    return res_all_df, agg_metrics_all_df

In [10]:
files_list = glob.glob('/home/user/New_Buildings_Datasets/Enernoc/csv-only/processed/*.csv')

dataset = 'Enernoc'
os.makedirs(f'forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    # if results is not None:
    #     results.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)
    print('')

2024-10-23 11:18:02.056044 /home/user/New_Buildings_Datasets/Enernoc/csv-only/processed/enernoc.csv
2024-10-23 11:18:02.168482 (8785, 100)
2024-10-23 11:18:02.169384 0 / 100 767
2024-10-23 11:18:10.710956 1 / 100 304
2024-10-23 11:18:16.570383 2 / 100 399
2024-10-23 11:18:22.244974 3 / 100 21
2024-10-23 11:18:27.711984 4 / 100 805
2024-10-23 11:18:33.231422 Saving...
2024-10-23 11:18:33.401758 5 / 100 14
2024-10-23 11:18:38.825496 6 / 100 404
2024-10-23 11:18:44.469694 7 / 100 78
2024-10-23 11:18:49.949308 8 / 100 731
2024-10-23 11:18:55.514829 9 / 100 218
2024-10-23 11:19:01.135589 Saving...
2024-10-23 11:19:01.481794 10 / 100 366
2024-10-23 11:19:07.186458 11 / 100 766
2024-10-23 11:19:12.762149 12 / 100 197
2024-10-23 11:19:18.397972 13 / 100 30
2024-10-23 11:19:24.134988 14 / 100 742
2024-10-23 11:19:29.652941 Saving...
2024-10-23 11:19:30.180227 15 / 100 32
2024-10-23 11:19:35.813991 16 / 100 137
2024-10-23 11:19:41.350888 17 / 100 36
2024-10-23 11:19:46.904208 18 / 100 9
2024-10-

In [16]:
torch.cuda.empty_cache()